![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/annotation/text/english/sentence-embeddings/InstructorEmbeddings.ipynb)

## Colab Setup

In [ ]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

In [ ]:
import sparknlp

from sparknlp.base import *
from sparknlp.common import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd

# Comment out this line  and uncomment the next one to enable GPU mode and High RAM
# 
spark = sparknlp.start()

# spark = sparknlp.start(gpu=True)

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

# Download InstructorEmbedding Model and Create Spark NLP Pipeline

Lets create a Spark NLP pipeline with the following stages:

In [ ]:
document_assembler = DocumentAssembler() \
            .setInputCol("text") \
            .setOutputCol("documents")

instruction = InstructorEmbeddings.pretrained(name='instructor_base', lang='en') \
            .setInstruction("Represent the Wikipedia document for retrieval: ") \
            .setInputCols(["documents"]) \
            .setOutputCol("instructor")

# Build pipeline with BART
pipe_components = [document_assembler, instruction]
pipeline = Pipeline().setStages( pipe_components)

Lets create a dataframe with some text to be annotated.

In [4]:
data = spark.createDataFrame([
            [1, """Capitalism has been dominant in the Western world since the end of feudalism, but most feel[who?] that 
            the term "mixed economies" more precisely describes most contemporary economies, due to their containing both 
            private-owned and state-owned enterprises. In capitalism, prices determine the demand-supply scale. For 
            example, higher demand for certain goods and services lead to higher prices and lower demand for certain 
            goods lead to lower prices. """]]).toDF("id", "text")

Extract the embeddings from the embeddings layer.

In [5]:
results = pipeline.fit(data).transform(data)
results.select("instructor.embeddings").show(truncate=False)

23/06/07 12:28:56 WARN MemoryStore: Not enough space to cache broadcast_1 in memory! (computed 4.6 GiB so far)
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------